In [1]:
!pip install pypsa
!pip install neptune-client
!pip install gymnasium

In [2]:
def calculate_offset_k_initialization(network_file, input_dir, k_method='mean', k_samples=1000, **env_kwargs):
  """
  Calculate the offset k for replacement reward method.
  Creates a temporary environment to perform the calculation.

  Parameters:
  -----------
  network_file : str
      Path to the PyPSA network file
  input_dir : str
      Directory containing constraint mappings
  k_method : str
      Method to calculate k: 'mean', 'worst_case', or 'percentile'
  k_samples : int
      Number of random samples to use for estimation
  **env_kwargs : dict
      Additional keyword arguments for environment creation

  Returns:
  --------
  float: Offset value k
  """
  print(f"Sampling {k_samples} random states to calculate offset k...")

  # Create temporary environment without offset_k (uses default)
  temp_env = Env2Gen1LoadConstrReplacement(network_file=network_file, input_dir=input_dir, **env_kwargs)
  #this initializes episode_length to number of snapshots and constraint_penalty_factor to None.
  #I'm just making this env to access certain attributes/ methods; which should be fine since none of these attributes/methods reference these two parameters.

  action_dim = temp_env.action_space.shape[0]

  objective_values = []
  successful_samples = 0
  try:
        for i in range(k_samples):
            try:
                # Reset environment to start fresh
                temp_env.reset(seed=42 + i)  # Use different seeds for variety
                # Sample random action
                random_action = temp_env.action_space.sample()  # This ensures [0,1] range
                # Take step - this handles all action scaling and application
                # make sure to get snapshot before the step to use when call _evaluate_stored_objective() again
                #i increase the current snapshot_idx by executing step() if do this line after step, when _evaluate_stored_objective() is run it evaluates the objective for the next step!
                current_snapshot = pd.Timestamp(temp_env.network.snapshots[temp_env.snapshot_idx]).isoformat()
                obs, reward, terminated, truncated, info = temp_env.step(random_action)
                # Get the base objective value (the -J(s) part, before any penalties or offsets)
                obj_value = temp_env._evaluate_stored_objective(current_snapshot)
                objective_values.append(obj_value)
                successful_samples += 1
                #print(info["constraint_violations"])

                # Progress indicator every 200 samples
                if (i + 1) % 200 == 0:
                    print(f"  Completed {i + 1}/{k_samples} samples...")

            except Exception as e:
                # Skip failed samples but continue
                if i < 5:  # Only print first few errors to avoid spam
                    print(f"  Sample {i} failed: {e}")
                continue

        # Calculate offset based on method
        if objective_values:
            if k_method == 'worst_case':
                k = abs(max(objective_values))
                print(f"  Using worst-case method: k = |{max(objective_values):.2f}| = {k:.2f}")
            else:  # method == 'mean'
                mean_val = np.mean(objective_values)
                k = abs(mean_val)
                print(f"  Using mean method: k = |{mean_val:.2f}| = {k:.2f}")

            print(f"  Successfully sampled {successful_samples}/{k_samples} states")
            print(f"  Objective value range: [{min(objective_values):.2f}, {max(objective_values):.2f}]")
        else:
            print("  Warning: No successful samples, using default k value")
            k = 2500  # Default fallback value

  except Exception as e:
          print(f"Error in offset calculation: {e}")
          import traceback
          traceback.print_exc()
          k = 2500  # Default fallback value
  return k

In [3]:
#Install debugger
!pip install ipdb
import ipdb

In [4]:
import pypsa
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import gc
import psutil
import matplotlib.pyplot as plt

import neptune

from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

[neptune] [warning] NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs-legacy.neptune.ai/setup/upgrading/


In [5]:
import os
import pickle
import numpy as np

# Define a placeholder class for PicklableLinearExpr
# This is needed because the constraints were pickled with this class definition
# and it's not directly available in the main namespace during unpickling.
# The actual content might be more complex depending on how it was originally defined,
# but a basic structure might be enough for pickle to load the object references.
class PicklableLinearExpr:
    def __init__(self, vars, coeffs, const):
        # Assuming these are the key attributes needed for unpickling
        self.vars = np.asarray(vars) if vars is not None else np.array([])
        self.coeffs = np.asarray(coeffs) if coeffs is not None else np.array([])
        self.const = const if const is not None else 0.0

    # Add __setstate__ and __getstate__ if needed for custom pickling logic
    # based on how the original class handled it. A basic __init__ might suffice.

def load_dictionary(network_file, dict_name,input_dir="snapshot_dicts"):
    """
    Load previously saved variable ID to name mapping from a file.

    Parameters:
    -----------
    network_file : str
        Path to the network file used to create the mapping
    input_dir : str, optional
        Directory where the mapping file is stored

    Returns:
    --------
    dict : var_id_to_name - The loaded variable ID to name mapping
    """

    # Get the network filename without path or extension
    network_name = os.path.basename(network_file)
    network_name = os.path.splitext(network_name)[0]

    # Create input filename
    var_map_file = os.path.join(input_dir, f"{network_name}_{dict_name}.pkl")

    # Check if file exists
    if not os.path.exists(var_map_file):
        raise FileNotFoundError(f"Variable mapping file for {network_name} not found in {input_dir}")

    # Load mapping from file
    with open(var_map_file, 'rb') as f:
        var_id_to_name = pickle.load(f)

    print(f"Loaded variable mapping from: {var_map_file}")

    return var_id_to_name

In [ ]:
def fix_artificial_lines_reasonable(network):
    """
    Fix artificial lines with reasonable capacity values:
    - s_nom = based on connected bus demand (with safety factor)
    - s_nom_extendable = False (non-extendable)
    - Keep capacity high enough to meet demand
    """
    print("=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===")
    
    # Find artificial lines
    artificial_lines = [line for line in network.lines.index
                       if any(keyword in str(line).lower() for keyword in ['new', '<->', 'artificial'])]
    
    if not artificial_lines:
        # If no artificial lines found by name, look for lines with s_nom=0
        # which is often a sign of artificial lines
        zero_capacity_lines = network.lines[network.lines.s_nom == 0].index.tolist()
        if zero_capacity_lines:
            artificial_lines = zero_capacity_lines
    
    print(f"Found {len(artificial_lines)} artificial lines to fix:")
    
    # Get maximum demand per bus across all snapshots
    bus_max_demand = {}
    for bus in network.buses.index:
        bus_demand = 0
        for load_name, load in network.loads.iterrows():
            if load.bus == bus and load_name in network.loads_t.p_set.columns:
                bus_demand = max(bus_demand, network.loads_t.p_set[load_name].max())
        bus_max_demand[bus] = bus_demand
    
    # Fix each artificial line with reasonable capacity
    for line_name in artificial_lines:
        # Get connected buses
        bus0 = network.lines.loc[line_name, 'bus0']
        bus1 = network.lines.loc[line_name, 'bus1']
        
        # Get maximum demand at these buses
        bus0_demand = bus_max_demand.get(bus0, 0)
        bus1_demand = bus_max_demand.get(bus1, 0)
        
        # Calculate required capacity with safety factor
        # Use 3x the higher demand to ensure adequate capacity
        safety_factor = 3.0
        required_capacity = max(bus0_demand, bus1_demand) * safety_factor
        
        # Ensure minimum reasonable capacity (1000 MW)
        required_capacity = max(required_capacity, 1000)
        
        print(f"\n🔧 Fixing: {line_name}")
        print(f"    Connected buses: {bus0} ↔ {bus1}")
        print(f"    Bus demands: {bus0}: {bus0_demand:.1f} MW, {bus1}: {bus1_demand:.1f} MW")
        
        # Set s_nom to required capacity
        old_s_nom = network.lines.loc[line_name, 's_nom']
        network.lines.loc[line_name, 's_nom'] = required_capacity
        print(f"    s_nom: {old_s_nom} → {required_capacity:.1f} MW")
        
        # Make sure line is not extendable
        if 's_nom_extendable' not in network.lines.columns:
            network.lines['s_nom_extendable'] = False
        network.lines.loc[line_name, 's_nom_extendable'] = False
        print(f"    s_nom_extendable: → False")
    
    return network

def create_pypsa_network(network_file):
    """Create a PyPSA network from the .nc file."""
    # Initialize network
    network = pypsa.Network(network_file)
    for storage_name in network.storage_units.index:
        # Use .loc for direct assignment to avoid SettingWithCopyWarning
        network.storage_units.loc[storage_name, 'cyclic_state_of_charge'] = False

    fix_artificial_lines_reasonable(network)

    return network

In [7]:
def get_variable_value(network, var_name):
    """
    Get the current value of a single optimization variable from the network.

    Parameters:
    -----------
    network : pypsa.Network
        The PyPSA network object
    var_name : str
        Variable name like 'Generator-p[snapshot=now,Generator=coal_gen_1]'

    Returns:
    --------
    float : current value of the variable
    """
    # Parse the variable name
    # Format: ComponentName-attribute[dimension1=value1,dimension2=value2,...]

    # Split on first '[' to separate base name from coordinates
    if '[' in var_name:
        base_name, coords_str = var_name.split('[', 1)
        coords_str = coords_str.rstrip(']')  # Remove trailing ']'
    else:
        base_name = var_name
        coords_str = ""

    # Parse base name: ComponentName-attribute
    component_name, attribute = base_name.split('-', 1)

    # Parse coordinates if they exist
    coords = {}
    if coords_str:
        # Split by comma and parse key=value pairs
        for coord_pair in coords_str.split(','):
            key, value = coord_pair.split('=', 1)
            coords[key.strip()] = value.strip()

    # Determine if this has time dimension (snapshot)
    has_snapshot = 'snapshot' in coords

    if has_snapshot:
        # Access dynamic dataframe using n.dynamic()
        snapshot_value = coords['snapshot']
        component_instance = coords[component_name]


        # Special handling for branch flow variables
        if component_name in network.passive_branch_components and attribute == 's':
            # For branch components, 's' is stored as 'p0' in the network
            # We can use p0 directly as the value of 's'
            try:
                return network.dynamic(component_name)['p0'].loc[snapshot_value, component_instance]
            except Exception as e:
                print(f"Warning: Could not get flow value for {var_name}: {e}")
                return 0.0

        # Get dynamic data for normal case
        dynamic_data = network.dynamic(component_name)

        # Access the specific attribute DataFrame
        if attribute in dynamic_data:
            return dynamic_data[attribute].loc[snapshot_value, component_instance]
        else:
            raise KeyError(f"Attribute {attribute} not found in dynamic data for {component_name}")

    else:
        # Access static dataframe using n.static()
        component_instance = coords[component_name]

        # Get static data
        static_data = network.static(component_name)

        # Access the value
        return static_data.loc[component_instance, attribute]

def evaluate_objective(variable_names_dict,var_indices_dict, network, snapshot_idx):
        """
        Direct evaluation without mock objects.
        Only includes terms from the current snapshot in the objective function.
        """
        ipdb.set_trace()
        temp_model = network.optimize.create_model()

        # Extract objective components
        obj_expr = temp_model.objective
        objective_coeffs = obj_expr.coeffs.copy()
        coeffs_flat = objective_coeffs.values.flatten()
        objective_const = obj_expr.const.copy() if hasattr(obj_expr, 'const') else 0
        # Get the current snapshot name
        ipdb.set_trace()
        snapshot_name = pd.Timestamp(network.snapshots[snapshot_idx]).isoformat()
        ipdb.set_trace()
        result = np.sum(coeffs_flat[var_indices_dict[snapshot_name]] *
                        [get_variable_value(network,var_name) for var_name in variable_names_dict[snapshot_name]]) + objective_const
        ipdb.set_trace()
        return result

In [8]:
def evaluate_baseline_reward(network_file, env,input_dir):
    network_baseline=create_pypsa_network(network_file)#network to run pypsa optimize on
    network_baseline.optimize()
    snapshots=network_baseline.snapshots
    variable_names_dict=load_dictionary(network_file=network_file, dict_name="variable_names_dict",input_dir=input_dir)
    var_indices_dict=load_dictionary(network_file=network_file, dict_name="var_indices_dict",input_dir=input_dir)
    objective_sum=0
    for snapshot_idx in range(len(snapshots)):
        objective_sum+= evaluate_objective(variable_names_dict,var_indices_dict, network_baseline,snapshot_idx)
    baseline_reward_value=-objective_sum*(env.episode_length/ len(snapshots))#assuming episode length is a multiple of the number of snapshots
    #reward is -1 times the objective value
    return baseline_reward_value

In [9]:
import ipdb
class Env2Gen1LoadConstr(gym.Env):
    """
    OpenAI Gym environment for Optimal Power Flow using PyPSA.
    Enhanced to handle dispatchable generators, renewable generators, and storage units.

    Action Space: Continuous setpoints for all controllable components within their capacity limits
    - Dispatchable generators: scaled between p_min_pu*p_nom and p_max_pu*p_nom
    - Renewable generators: scaled between 0 and current p_max_pu*p_nom (time-varying)
    - Storage units: scaled between -p_nom (charging) and +p_nom (discharging)
    (This follows http://arxiv.org/abs/2403.17831.)
    """

    def __init__(self,network_file, episode_length_factor=None, constraint_penalty_factor=100, input_dir="var_constraint_map"):
        super().__init__()

        self.network_file = network_file # Store network file path
        self.input_dir=input_dir

        # Use provided network or create new one
        self.network =create_pypsa_network(network_file)

        self._initialize_optimization_components()
        self.penalty_factor=constraint_penalty_factor
        self.reward_method = "summation" # Default reward method for the base class

        # Episode management
        self.total_snapshots = len(self.network.snapshots)
        self.episode_length = episode_length_factor*self.total_snapshots if episode_length_factor is not None else self.total_snapshots
        self.current_step = 0  # Steps within current episode
        self.snapshot_idx = 0  # Current snapshot index (cycles through all snapshots)

        # Initialize component categorization
        self._categorize_components()

        # Create action space
        self._create_action_space()

        # Initialize the network state
        self.reset()

        # Create observation space
        low_bounds, high_bounds = self.create_observation_bounds()
        self.observation_space = spaces.Box(
            low=low_bounds,
            high=high_bounds,
            dtype=np.float32
        )
        #TO DO: If use .pf instead of .lpf, add another of each term for active power AND reactive power

    def _initialize_optimization_components(self):
        """
        Initialize all optimization components in one pass to avoid creating multiple models.
        This method:
        1. Creates the optimization model once
        2. Extracts objective components (vars, coeffs, const)
        3. Creates the variable ID to name mapping
        4. Extracts constraints
        5. Cleans up the model
        """
        # Create model once - this is an expensive operation
        temp_model = self.network.optimize.create_model()

        # Extract objective components
        obj_expr = temp_model.objective
        objective_coeffs = obj_expr.coeffs.copy()
        self.coeffs_flat = objective_coeffs.values.flatten()
        self.objective_const = obj_expr.const.copy() if hasattr(obj_expr, 'const') else 0

        self.variable_names_dict= load_dictionary(network_file=self.network_file, dict_name="variable_names_dict", input_dir=self.input_dir)
        self.var_indices_dict= load_dictionary(network_file=self.network_file, dict_name="var_indices_dict", input_dir=self.input_dir)

        # Clean up to free memory
        del temp_model, obj_expr
        gc.collect()

    def _categorize_components(self):
        """
        Categorize generators and identify storage units for action space.
        """
        # Get generators with time-varying p_max_pu (renewable generators)
        renewable_gens = self.network.generators_t.p_max_pu.columns

        slack_generators = self.network.generators[self.network.generators.control == "Slack"].index
        # in the 10-node SA network there are 4 slack gens so this should return a list of indexes

        # Dispatchable generators: not slack, not renewable
        self.dispatchable_gens = self.network.generators[
            (~self.network.generators.index.isin(slack_generators)) &
            (~self.network.generators.index.isin(renewable_gens))
        ].index

        # Renewable generators: have time-varying p_max_pu, not slack
        self.renewable_gens = self.network.generators[
            (self.network.generators.index.isin(renewable_gens)) &
            (~self.network.generators.index.isin(slack_generators))
        ].index

        # Storage units (if any exist in the network)
        self.storage_units = self.network.storage_units.index

        # Store names as lists for easier indexing
        self.dispatchable_names = list(self.dispatchable_gens)
        self.renewable_names = list(self.renewable_gens)
        self.storage_names = list(self.storage_units)

        # Store counts
        self.n_dispatchable = len(self.dispatchable_names)
        self.n_renewable = len(self.renewable_names)
        self.n_storage = len(self.storage_names)

        # Get static limits for dispatchable generators
        if self.n_dispatchable > 0:
            dispatchable_df = self.network.generators.loc[self.dispatchable_gens]
            self.disp_p_min = (dispatchable_df.p_min_pu * dispatchable_df.p_nom).values#returns numpy arrays
            self.disp_p_max = (dispatchable_df.p_max_pu * dispatchable_df.p_nom).values
        else:
            self.disp_p_min = np.array([])
            self.disp_p_max = np.array([])

        # Get nominal capacities and minimum limits for renewable generators
        if self.n_renewable > 0:
            renewable_df = self.network.generators.loc[self.renewable_gens]
            self.renewable_p_nom = renewable_df.p_nom.values
            self.renewable_p_min_pu = renewable_df.p_min_pu.values
        else:
            self.renewable_p_nom = np.array([])
            self.renewable_p_min_pu = np.array([])

        # Get storage unit capacities
        if self.n_storage > 0:
            storage_df = self.network.storage_units.loc[self.storage_units]
            #this is a bit redundant since self.storage_units is the array of all indices of self.network.storage_units but leave it in so could replace which indices you want
            self.storage_p_nom = storage_df.p_nom.values
        else:
            self.storage_p_nom = np.array([])


    def _create_action_space(self):
        """
        Create action space with four distinct parts:
        1. Dispatchable generators: [0,1] scaled to [p_min, p_max]
        2. Renewable generators: [0,1] scaled to [0, current_p_max_pu * p_nom]
        3. Storage p_set: [0,1] scaled to [-p_nom, +p_nom] (negative=charging, positive=discharging)
        4. Storage p_dispatch: [0,1] scaled to [0, p_nom] (discharging magnitude)
        """
        total_actions = self.n_dispatchable + self.n_renewable + (2 * self.n_storage)  # 2 actions per storage unit
        self.action_space = gym.spaces.Box(0, 1, shape=(total_actions,))

        # Store action space structure for easy reference
        self.action_structure = {
            'dispatchable': {
                'start': 0,
                'end': self.n_dispatchable,
                'count': self.n_dispatchable
            },
            'renewable': {
                'start': self.n_dispatchable,
                'end': self.n_dispatchable + self.n_renewable,
                'count': self.n_renewable
            },
            'storage_p_set': {
                'start': self.n_dispatchable + self.n_renewable,
                'end': self.n_dispatchable + self.n_renewable + self.n_storage,
                'count': self.n_storage
            },
            'storage_p_dispatch': {
                'start': self.n_dispatchable + self.n_renewable + self.n_storage,
                'end': self.n_dispatchable + self.n_renewable + (2 * self.n_storage),
                'count': self.n_storage
            }
        }

    def _get_storage_observation(self):
        """
        Get current storage unit states for observation.
        Returns previous SOC (normalized) and current inflow (normalized) for each storage unit.
        """
        if self.n_storage == 0:
            return np.array([])

        current_snapshot = self.network.snapshots[self.snapshot_idx]
        storage_obs = []

        for storage_name in self.storage_names:
            # 1. Previous State of Charge (using your exact logic)
            if self.snapshot_idx == 0:
                # For first snapshot, previous SOC is the initial value
                soc_prev = self.network.storage_units.state_of_charge_initial.loc[storage_name]
            else:
                previous_snapshot = self.network.snapshots[self.snapshot_idx - 1]
                soc_prev = self.network.storage_units_t.state_of_charge.loc[previous_snapshot, storage_name]

            # Get SOC limit from PyPSA parameters
            p_nom = self.network.storage_units.loc[storage_name, 'p_nom']
            max_hours = self.network.storage_units.loc[storage_name, 'max_hours']
            max_soc = p_nom * max_hours  # SOC limit (energy capacity)

            # Normalize SOC by its maximum possible value
            normalized_soc_prev = soc_prev / max_soc if max_soc > 0 else 0
            storage_obs.append(normalized_soc_prev)

            # 2. Current Inflow (normalized by p_nom)
            if hasattr(self.network.storage_units_t, 'inflow') and storage_name in self.network.storage_units_t.inflow.columns:
                current_inflow = self.network.storage_units_t.inflow.loc[current_snapshot, storage_name]
                # Normalize by p_nom for consistent scaling
                normalized_inflow = current_inflow / p_nom if p_nom > 0 else 0
            else:
                # If no inflow data exists, use zero
                normalized_inflow = 0.0
            storage_obs.append(normalized_inflow)

        return np.array(storage_obs, dtype=np.float32)

    def create_storage_observation_bounds(self):
        """
        Create bounds for storage unit observations.
        Since no spill: SOC bounds are always [0, 1] when normalized.
        """
        if self.n_storage == 0:
            return np.array([]), np.array([])

        values_per_storage = 2  # SOC + inflow
        total_storage_obs = self.n_storage * values_per_storage

        low_bounds = np.zeros(total_storage_obs)
        high_bounds = np.zeros(total_storage_obs)

        for i, storage_name in enumerate(self.storage_names):
            base_idx = i * values_per_storage

            # SOC bounds: Always [0, 1] when normalized by (p_nom * max_hours)
            low_bounds[base_idx] = 0.0      # Normalized SOC min
            high_bounds[base_idx] = 1.0     # Normalized SOC max

            # Inflow bounds: Get from historical data
            if hasattr(self.network.storage_units_t, 'inflow'):
                p_nom = self.network.storage_units.loc[storage_name, 'p_nom']
                if storage_name in self.network.storage_units_t.inflow.columns:
                    inflow_data = self.network.storage_units_t.inflow[storage_name]

                    min_inflow_norm = inflow_data.min() / p_nom if p_nom > 0 else 0
                    max_inflow_norm = inflow_data.max() / p_nom if p_nom > 0 else 0

                    low_bounds[base_idx + 1] = min_inflow_norm
                    high_bounds[base_idx + 1] = max_inflow_norm
                else:
                     # If inflow data exists but not for this specific storage unit
                    low_bounds[base_idx + 1] = 0.0
                    high_bounds[base_idx + 1] = 0.0
            else:
                # No inflow data
                low_bounds[base_idx + 1] = 0.0
                high_bounds[base_idx + 1] = 0.0

        return low_bounds.astype(np.float32), high_bounds.astype(np.float32)

    def create_observation_bounds(self):
        """
        Create bounds for the observation space based on:
        - Load p_set values
        - Renewable generator p_max_pu values
        - Storage unit previous SOC (normalized) and current inflow (normalized)
        """
        # 1. Load bounds
        load_p_set_all = self.network.loads_t.p_set  # DataFrame with all snapshots and loads
        load_low_bounds = load_p_set_all.min(axis=0).values  # Min across all snapshots for each load
        load_high_bounds = load_p_set_all.max(axis=0).values  # Max across all snapshots for each load

        # 2. Renewable generator bounds
        if self.n_renewable > 0:
            renewable_p_max_pu_all = self.network.generators_t.p_max_pu[self.renewable_names]
            renewable_low_bounds = renewable_p_max_pu_all.min(axis=0).values
            renewable_high_bounds = renewable_p_max_pu_all.max(axis=0).values
        else:
            renewable_low_bounds = np.array([])
            renewable_high_bounds = np.array([])

        # 3. Storage bounds (previous SOC + current inflow)
        storage_low_bounds, storage_high_bounds = self.create_storage_observation_bounds()

        # 4. Combine all bounds
        low_bounds = np.concatenate([load_low_bounds, renewable_low_bounds, storage_low_bounds])
        high_bounds = np.concatenate([load_high_bounds, renewable_high_bounds, storage_high_bounds])

        return low_bounds.astype(np.float32), high_bounds.astype(np.float32)

    def _get_observation(self):
        """
        Get current network state as observation.

        Returns observation vector with structure:
        [load_1_demand, load_2_demand, ..., load_n_demand,
        renewable_1_p_max_pu, renewable_2_p_max_pu, ..., renewable_m_p_max_pu,
        storage_1_prev_soc_norm, storage_1_current_inflow_norm,
        storage_2_prev_soc_norm, storage_2_current_inflow_norm,
        ...,
        storage_k_prev_soc_norm, storage_k_current_inflow_norm]
        """
        # 1. Load demands (dynamic values at current snapshot)
        load_demands = self.network.loads_t.p_set.iloc[self.snapshot_idx].values

        # 2. Renewable generator p_max_pu values (time-varying availability at current snapshot)
        if self.n_renewable > 0:
            renewable_p_max_pu = self.network.generators_t.p_max_pu.iloc[self.snapshot_idx][self.renewable_names].values
        else:
            renewable_p_max_pu = np.array([])

        # 3. Storage states (previous SOC normalized + current inflow normalized)
        storage_states = self._get_storage_observation()

        # 4. Combine all observations
        observation = np.concatenate([load_demands, renewable_p_max_pu, storage_states])

        return observation.astype(np.float32)

    def reset_network(self):
        """Reset and ensure essential DataFrames exist."""
        #Note that we do not just create a new network here, as this consumes more memory and previously led to a segmentation fault

        # Initialize/reset generators_t.p_set
        if not hasattr(self.network.generators_t, 'p_set') or self.network.generators_t.p_set.empty:
            self.network.generators_t.p_set = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.generators.index
            )
        else:
            self.network.generators_t.p_set.iloc[:, :] = 0.0

        # Initialize/reset storage_units_t attributes
        if not hasattr(self.network.storage_units_t, 'p_set') or self.network.storage_units_t.p_set.empty:
            self.network.storage_units_t.p_set = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.storage_units.index
            )
        else:
            self.network.storage_units_t.p_set.iloc[:, :] = 0.0


        if not hasattr(self.network.storage_units_t, 'p_dispatch') or self.network.storage_units_t.p_dispatch.empty:
            self.network.storage_units_t.p_dispatch = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.storage_units.index
            )
        else:
            self.network.storage_units_t.p_dispatch.iloc[:, :] = 0.0

        if not hasattr(self.network.storage_units_t, 'p_store') or self.network.storage_units_t.p_store.empty:
            self.network.storage_units_t.p_store = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.storage_units.index
            )
        else:
            self.network.storage_units_t.p_store.iloc[:, :] = 0.0

        if not hasattr(self.network.storage_units_t, 'state_of_charge') or self.network.storage_units_t.state_of_charge.empty:
            self.network.storage_units_t.state_of_charge = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.storage_units.index
            )
        else:
            self.network.storage_units_t.state_of_charge.iloc[:, :] = 0.0

        if not hasattr(self.network.storage_units_t, 'spill') or self.network.storage_units_t.spill.empty:
            self.network.storage_units_t.spill = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.storage_units.index
            )
        else:
            self.network.storage_units_t.spill.iloc[:, :] = 0.0



    def reset(self, seed=None, options=None):
        """
        Reset the environment to initial state.
        Returns initial observation and info (gymnasium format).
        """
        # Set seed if provided
        if seed is not None:
            self.seed(seed)

        # Reset episode counters
        self.current_step = 0
        self.snapshot_idx = 0

        self.reset_network()
        #self._test_hidden_references()
        #can run that instead to check if there are hidden references preventing network cleanup

        # Get initial observation
        obs = self._get_observation()

        # Info dict for gymnasium compatibility
        info = {
            'current_step': self.current_step,
            'snapshot_idx': self.snapshot_idx
        }

        return obs, info

    def scale_action(self, action):
        """
        Scale action from [0,1] range to appropriate ranges for each component type.

        Parameters:
        -----------
        action : numpy.ndarray
            Action values in range [0,1] for all components

        Returns:
        --------
        dict: Scaled actions organized by component type
        """
        scaled_actions = {}

        # Scale dispatchable generator actions
        if self.n_dispatchable > 0:
            disp_actions = action[self.action_structure['dispatchable']['start']:
                                self.action_structure['dispatchable']['end']]
            scaled_actions['dispatchable'] = self.disp_p_min + disp_actions * (self.disp_p_max - self.disp_p_min)
        else:
            scaled_actions['dispatchable'] = np.array([])

        # Scale renewable generator actions (time-dependent scaling)
        if self.n_renewable > 0:
            renewable_actions = action[self.action_structure['renewable']['start']:
                                     self.action_structure['renewable']['end']]

            # Get current p_max_pu values for this snapshot
            current_p_max_pu = self.network.generators_t.p_max_pu.iloc[self.snapshot_idx][self.renewable_names].values
            current_p_max = current_p_max_pu * self.renewable_p_nom
            current_p_min = self.renewable_p_min_pu * self.renewable_p_nom

            # Scale from [0,1] to [current_p_min, current_p_max]
            scaled_actions['renewable'] = current_p_min + renewable_actions * (current_p_max - current_p_min)
        else:
            scaled_actions['renewable'] = np.array([])

        # Scale storage p_set actions
        if self.n_storage > 0:
            storage_p_set_actions = action[self.action_structure['storage_p_set']['start']:
                                         self.action_structure['storage_p_set']['end']]

            # Scale from [0,1] to [-p_nom, +p_nom]
            # 0 -> -p_nom (full charging), 0.5 -> 0 (no net power), 1 -> +p_nom (full discharging)
            scaled_actions['storage_p_set'] = -self.storage_p_nom + 2 * storage_p_set_actions * self.storage_p_nom

            # Scale storage p_dispatch actions
            storage_p_dispatch_actions = action[self.action_structure['storage_p_dispatch']['start']:
                                              self.action_structure['storage_p_dispatch']['end']]

            # Scale from [0,1] to [0, p_nom]
            # 0 -> 0 (no discharge), 1 -> p_nom (full discharge)
            scaled_actions['storage_p_dispatch'] = storage_p_dispatch_actions * self.storage_p_nom
        else:
            scaled_actions['storage_p_set'] = np.array([])
            scaled_actions['storage_p_dispatch'] = np.array([])

        return scaled_actions

    def _update_storage_soc_single_snapshot(self, storage_name):
        if self.snapshot_idx == 0:
            # For first snapshot, previous SOC is the initial value
            soc_prev = self.network.storage_units.state_of_charge_initial.loc[storage_name]
        else:
            previous_snapshot = self.network.snapshots[self.snapshot_idx - 1]
            soc_prev = self.network.storage_units_t.state_of_charge.loc[previous_snapshot, storage_name]

        current_snapshot = self.network.snapshots[self.snapshot_idx]

        #Get storage parameters
        storage_unit = self.network.storage_units.loc[storage_name]
        soc_max = storage_unit.p_nom * storage_unit.max_hours
        eff_store = storage_unit.efficiency_store
        eff_dispatch = storage_unit.efficiency_dispatch
        standing_loss = storage_unit.standing_loss

        # Get time step
        if hasattr(self.network.snapshot_weightings, 'stores'):
            delta_t = self.network.snapshot_weightings.stores.iloc[self.snapshot_idx]
        else:
            delta_t = self.network.snapshot_weightings.iloc[self.snapshot_idx]

        eff_standing = (1 - standing_loss) ** delta_t

        # Get current operations (these determine the SOC change)
        p_store = self.network.storage_units_t.p_store.loc[current_snapshot, storage_name]
        p_dispatch = self.network.storage_units_t.p_dispatch.loc[current_snapshot, storage_name]
        if storage_name in self.network.storage_units_t.inflow.columns:
          inflow = self.network.storage_units_t.inflow.loc[current_snapshot, storage_name]
        else:
          inflow=0

        # Calculate SOC without spill (could be non-zero even if soc_prev=0)
        soc_without_spill = (soc_prev * eff_standing +
                            (p_store * eff_store - p_dispatch/eff_dispatch + inflow) * delta_t)

        # Calculate required spill
        required_spill = max(0, (soc_without_spill - soc_max) / delta_t)

        # Final SOC after spill
        soc_actual = min(soc_without_spill, soc_max)

        # Update the network
        self.network.storage_units_t.state_of_charge.loc[current_snapshot, storage_name] = soc_actual
        if hasattr(self.network, 'storage_units_t') and 'spill' in self.network.storage_units_t:
            self.network.storage_units_t.spill.loc[current_snapshot, storage_name] = required_spill

    def get_variable_value(self,var_name):
        """
        Get the current value of a single optimization variable from the network.

        Parameters:
        -----------
        network : pypsa.Network
            The PyPSA network object
        var_name : str
            Variable name like 'Generator-p[snapshot=now,Generator=coal_gen_1]'

        Returns:
        --------
        float : current value of the variable
        """
        # Parse the variable name
        # Format: ComponentName-attribute[dimension1=value1,dimension2=value2,...]

        # Split on first '[' to separate base name from coordinates
        if '[' in var_name:
            base_name, coords_str = var_name.split('[', 1)
            coords_str = coords_str.rstrip(']')  # Remove trailing ']'
        else:
            base_name = var_name
            coords_str = ""

        # Parse base name: ComponentName-attribute
        component_name, attribute = base_name.split('-', 1)

        # Parse coordinates if they exist
        coords = {}
        if coords_str:
            # Split by comma and parse key=value pairs
            for coord_pair in coords_str.split(','):
                key, value = coord_pair.split('=', 1)
                coords[key.strip()] = value.strip()

        # Determine if this has time dimension (snapshot)
        has_snapshot = 'snapshot' in coords

        if has_snapshot:
            # Access dynamic dataframe using n.dynamic()
            snapshot_value = coords['snapshot']
            component_instance = coords[component_name]


            # Special handling for branch flow variables
            if component_name in self.network.passive_branch_components and attribute == 's':
                # For branch components, 's' is stored as 'p0' in the network
                # We can use p0 directly as the value of 's'
                try:
                    return self.network.dynamic(component_name)['p0'].loc[snapshot_value, component_instance]
                except Exception as e:
                    print(f"Warning: Could not get flow value for {var_name}: {e}")
                    return 0.0

            # Get dynamic data for normal case
            dynamic_data = self.network.dynamic(component_name)

            # Access the specific attribute DataFrame
            if attribute in dynamic_data:
                return dynamic_data[attribute].loc[snapshot_value, component_instance]
            else:
                raise KeyError(f"Attribute {attribute} not found in dynamic data for {component_name}")

        else:
            # Access static dataframe using n.static()
            component_instance = coords[component_name]

            # Get static data
            static_data = self.network.static(component_name)

            # Access the value
            return static_data.loc[component_instance, attribute]


    def create_variable_values_mapping(self,variable_names):
        """
        Create a mapping from optimization variable names to their current values in the network.

        Parameters:
        -----------
        network : pypsa.Network
            The PyPSA network object
        variable_names : list
            List of variable names like ['Generator-p[snapshot=now,Generator=coal_gen_1]', ...]

        Returns:
        --------
        dict : mapping from variable name to current value
        """
        var_values = {}

        for var_name in variable_names:
            try:
                value = self.get_variable_value(var_name)
                var_values[var_name] = value
            except Exception as e:
                print(f"Warning: Could not get value for {var_name}: {e}")
                var_values[var_name] = 0.0  # Default fallback

        return var_values

    def _evaluate_stored_objective(self,snapshot_name):
        """
        Direct evaluation without mock objects.
        Only includes terms from the current snapshot in the objective function.
        """

        result = np.sum(self.coeffs_flat[self.var_indices_dict[snapshot_name]] *
                        [get_variable_value(self.network,var_name) for var_name in self.variable_names_dict[snapshot_name]]) + self.objective_const
        return result

    def _calculate_reward(self):
        """Calculate reward using stored objective components."""
        # Get the current snapshot name

        current_snapshot = pd.Timestamp(self.network.snapshots[self.snapshot_idx]).isoformat()
        return -1 * self._evaluate_stored_objective(current_snapshot)

    def calculate_constrained_reward(self):
        """
        Calculate reward using summation method with dynamic constraint checking.

        Summation method:
        - Reward = -J(s) - P(s)
        """
        try:
            # Get base reward from objective function (negative for minimization)
            base_reward = self._calculate_reward()

            current_snapshot = self.network.snapshots[self.snapshot_idx]

            # Initialize constraint tracking
            constraint_results = {
                'all_satisfied': True,
                'violations': {},
                'total_violation': 0.0,
                'violations_by_group': {}
            }

            # 1. Check slack generator constraints
            slack_generators = self.network.generators[self.network.generators.control == "Slack"].index
            if not slack_generators.empty:
                for gen_name in slack_generators:
                    # Get actual power output after power flow
                    p_actual = self.network.generators_t.p.loc[current_snapshot, gen_name]

                    # Get limits
                    p_min = self.network.generators.loc[gen_name, 'p_min_pu'] * self.network.generators.loc[gen_name, 'p_nom']
                    p_max = self.network.generators.loc[gen_name, 'p_max_pu'] * self.network.generators.loc[gen_name, 'p_nom']

                    # Check lower bound
                    if p_actual < p_min:
                        violation = float(p_min - p_actual)
                        constraint_name = f"Generator-slack-p-lower[snapshot={current_snapshot},Generator={gen_name}]"
                        constraint_results['violations'][constraint_name] = violation
                        constraint_results['total_violation'] += violation
                        constraint_results['all_satisfied'] = False

                    # Check upper bound
                    if p_actual > p_max:
                        violation = float(p_actual - p_max)
                        constraint_name = f"Generator-slack-p-upper[snapshot={current_snapshot},Generator={gen_name}]"
                        constraint_results['violations'][constraint_name] = violation
                        constraint_results['total_violation'] += violation
                        constraint_results['all_satisfied'] = False

            # 2. Check line flow constraints (CORRECTED)
            for line_name in self.network.lines.index:
                # Get line parameters
                s_nom = self.network.lines.loc[line_name, 's_nom']
                s_max_pu = 1.0  # Default, or get from lines_t.s_max_pu if it exists

                # Calculate active power limit (this is what PyPSA's linear constraints check)
                s_max = s_max_pu * s_nom

                # Get active power flow from the linear power flow
                # In PyPSA's linear formulation, this is the 's' variable value
                p0 = abs(self.network.lines_t.p0.loc[current_snapshot, line_name])

                # Check if active power flow exceeds limit
                if p0 > s_max:
                    violation = float(p0 - s_max)
                    constraint_name = f"Line-fix-s-upper[snapshot={current_snapshot},Line={line_name}]"
                    constraint_results['violations'][constraint_name] = violation
                    constraint_results['total_violation'] += violation
                    constraint_results['all_satisfied'] = False


            # Calculate penalty
            total_violation = float(constraint_results['total_violation'])
            penalty = self.penalty_factor * total_violation

            # Calculate final reward using summation method
            constrained_reward = base_reward - penalty

            # Ensure reward is a scalar
            if hasattr(constrained_reward, '__len__'):
                constrained_reward = float(constrained_reward)

            return constrained_reward, constraint_results

        except Exception as e:
            print(f"Error calculating summation reward: {e}")
            # Fall back to base reward on error
            return self._calculate_reward(), {
                'all_satisfied': True,
                'violations': {},
                'total_violation': 0.0
            }


    def step(self, action):
        """
        Execute one time step within the environment.

        Args:
            action: Array of setpoints for all controllable components [disp_gen1, disp_gen2, ...,
                   renewable_gen1, renewable_gen2, ..., storage1, storage2, ...]

        Returns:
            observation: Network state after action
            reward: Reward for this action
            terminated: Whether episode is finished due to task completion
            truncated: Whether episode is finished due to time limit
            info: Additional information
        """
        scaled_actions = self.scale_action(action)
        # Apply dispatchable generator setpoints
        if self.n_dispatchable > 0:
            for i, gen_name in enumerate(self.dispatchable_names):
                self.network.generators_t.p_set.iloc[self.snapshot_idx, self.network.generators_t.p_set.columns.get_loc(gen_name)] = scaled_actions['dispatchable'][i]

        # Apply renewable generator setpoints
        if self.n_renewable > 0:
            for i, gen_name in enumerate(self.renewable_names):
                self.network.generators_t.p_set.iloc[self.snapshot_idx,
                    self.network.generators_t.p_set.columns.get_loc(gen_name)] = scaled_actions['renewable'][i]

        # Apply storage unit setpoints
        if self.n_storage > 0:
            for i, storage_name in enumerate(self.storage_names):
                self.network.storage_units_t.p_set.iloc[self.snapshot_idx,
                    self.network.storage_units_t.p_set.columns.get_loc(storage_name)] = scaled_actions['storage_p_set'][i]
                self.network.storage_units_t.p_dispatch.iloc[self.snapshot_idx,
                    self.network.storage_units_t.p_dispatch.columns.get_loc(storage_name)] = scaled_actions['storage_p_dispatch'][i]
                self.network.storage_units_t.p_store.iloc[self.snapshot_idx,
                    self.network.storage_units_t.p_store.columns.get_loc(storage_name)] = scaled_actions['storage_p_dispatch'][i] - scaled_actions['storage_p_set'][i]

            # Update state of charge using PyPSA's energy balance equation
                if self.snapshot_idx > 0:
                    self._update_storage_soc_single_snapshot(storage_name)
        # Run power flow to get new network state
        try:
            self.network.lpf(self.network.snapshots[self.snapshot_idx])
            power_flow_converged = True
        except Exception as e:
            print(f"Power flow failed: {e}")
            power_flow_converged = False

        # Calculate reward using constrained reward function
        reward, constraint_results = self.calculate_constrained_reward()

        # Increment step counters
        self.current_step += 1
        self.snapshot_idx += 1

        # Handle cycling through snapshots
        if self.snapshot_idx >= self.total_snapshots:
            self.snapshot_idx = 0
            self.reset_network()

        # Get new observation
        observation = self._get_observation()

        # Check if episode is done
        episode_done = self._check_done()
        terminated = False
        truncated = episode_done

        # Additional info
        info = {
            'dispatchable_setpoints': scaled_actions['dispatchable'],
            'renewable_setpoints': scaled_actions['renewable'],
            'storage_p_set': scaled_actions['storage_p_set'],
            'storage_p_dispatch': scaled_actions['storage_p_dispatch'],
            'power_flow_converged': power_flow_converged,
            'dispatchable_names': self.dispatchable_names,
            'renewable_names': self.renewable_names,
            'storage_names': self.storage_names,
            'current_step': self.current_step,
            'snapshot_idx': self.snapshot_idx,
            'constraints_satisfied': constraint_results['all_satisfied'],
            'constraint_violations': constraint_results['violations'],
            'total_violation': constraint_results['total_violation']
        }

        return observation, reward, terminated, truncated, info

    def _check_done(self):
        """
        Check if episode should terminate.

        Episode terminates when we've reached the specified episode length.
        """
        if self.current_step >= self.episode_length:
            return True

        return False

    def seed(self, seed=None):
        """
        Set the random seed for reproducible experiments.
        """
        np.random.seed(seed)
        return [seed]

    def render(self, mode='human', info=None):
        """
        Render the environment state.

        Parameters:
        -----------
        mode : str
            Rendering mode (only 'human' supported)
        info : dict, optional
            Information dictionary from step() method containing constraint data
        """
        print("=== Current Network State ===")
        print(f"Episode step: {self.current_step}/{self.episode_length}")
        print(f"Snapshot index: {self.snapshot_idx}/{self.total_snapshots}")
        print(f"Current snapshot: {self.network.snapshots[self.snapshot_idx]}")
        print(f"Generator setpoints: {self.network.generators_t.p_set.iloc[self.snapshot_idx].values}")
        print(f"Load values: {self.network.loads_t.p_set.iloc[self.snapshot_idx].values}")

        all_satisfied = info['constraints_satisfied']
        total_violation = info['total_violation']
        violations = info['constraint_violations']


        print(f"All constraints satisfied: {all_satisfied}")
        print(f"Total constraint violation: {total_violation:.4f}")

        # Show violated constraints if any
        if not all_satisfied and violations:
            print("\n=== Constraint Violations ===")
            for constraint_name, violation in violations.items():
                print(f"  {constraint_name}: {violation:.4f}")

# network_file_path= "/Users/antoniagrindrod/Documents/pypsa-earth_project/pypsa-earth-RL/networks/elec_s_10_ec_lc1.0_1h.nc"
# input_dir="/Users/antoniagrindrod/Documents/pypsa-earth_project/pypsa-earth-RL/RL/var_constraint_map"
# replacement_reward_offset=calculate_offset_k_initialization(network_file=network_file_path, input_dir=input_dir)

In [10]:
class BackboneNetwork(nn.Module):
    def __init__(self, input_features, hidden_dimensions, out_features, dropout):
        super(BackboneNetwork, self).__init__()

        # SIMPLIFIED: Single hidden layer network for debugging
        self.neuralnet = nn.Sequential(
            nn.Linear(input_features, hidden_dimensions),
            nn.ReLU(),
            nn.Linear(hidden_dimensions, hidden_dimensions),
            nn.ReLU(),
            nn.Linear(hidden_dimensions, out_features)
        )

    def forward(self, x):
        output = self.neuralnet(x)
        return output

#Define the actor-critic network
class actorCritic(nn.Module):
    def __init__(self, actor, critic):
        super().__init__()
        self.actor = actor
        self.critic = critic
    def forward(self, state):
        action_pred = self.actor(state)
        value_pred = self.critic(state)
        return action_pred, value_pred
        #Returns both the action predictions and the value predictions.

#We'll use the networks defined above to create an actor and a critic. Then, we will create an agent, including the actor and the critic.
#finish this step later
# def create_agent(hidden_dimensions, dropout):
#     INPUT_FEATURES =env_train.
class PPO_agent:
    def __init__(self,
                 env,
                 device,
                 run,
                 hidden_dimensions,
                 dropout, discount_factor,
                 max_episodes,
                 print_interval,
                 PPO_steps,
                 n_trials,
                 epsilon,
                 entropy_coefficient,
                 learning_rate,
                 batch_size,
                 optimizer_name,
                 seed):

        self.seed = seed
        if seed is not None:
            # Set PyTorch seed for this class
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)

        self.env = env  # Store the environment as an attribute

        self.device = device
        self.run = run

        # Get observation and action space dimensions for gymnasium environment
        obs, _ = self.env.reset()

        self.INPUT_FEATURES = obs.shape[0]  # Flattened observation size
        self.ACTOR_OUTPUT_FEATURES = self.env.action_space.shape[0]* 2  # 2 parameters (alpha, beta) per action dimension

        self.HIDDEN_DIMENSIONS = hidden_dimensions

        self.CRITIC_OUTPUT_FEATURES = 1
        self.DROPOUT = dropout

        self.discount_factor = discount_factor
        self.max_episodes = max_episodes
        self.print_interval = print_interval
        self.PPO_steps=PPO_steps
        self.n_trials=n_trials
        self.epsilon=epsilon
        self.entropy_coefficient=entropy_coefficient
        self.learning_rate=learning_rate

        self.batch_size=batch_size

        # Initialize actor network
        self.actor = BackboneNetwork(
            self.INPUT_FEATURES, self.HIDDEN_DIMENSIONS, self.ACTOR_OUTPUT_FEATURES, self.DROPOUT
        ).to(self.device)

        # Initialize the final layer bias for Beta distribution
        for name, param in self.actor.named_parameters():
            if 'neuralnet.4.bias' in name:  # Adjust index based on your network structure
                # Initialize to produce alpha=beta=2 (uniform-like distribution centered at 0.5)
                param.data.fill_(0.0)  # softplus(0) + 1 = 2
                print(f"Initialized Beta parameters to produce uniform-like distribution")

        # Initialize critic network
        self.critic = BackboneNetwork(
            self.INPUT_FEATURES, self.HIDDEN_DIMENSIONS, self.CRITIC_OUTPUT_FEATURES, self.DROPOUT
        ).to(self.device)

        #Better move the .to(self.device) call separately for both self.actor and self.critic. This ensures the individual parts of the model are moved to the correct device before combined into the actorCritic class
        # Combine into a single actor-critic model
        self.model = actorCritic(self.actor, self.critic)

        try:
            # Try to get the optimizer from torch.optim based on the provided name
            self.optimizer = getattr(torch.optim, optimizer_name)(self.model.parameters(), lr=self.learning_rate)
        except AttributeError:
            # Raise an error if the optimizer_name is not valid
            raise ValueError(f"Optimizer '{optimizer_name}' is not available in torch.optim.")

    def calculate_returns(self, rewards):
        returns = []
        cumulative_reward = 0
        for r in reversed(rewards):
            cumulative_reward = r +cumulative_reward*self.discount_factor
            returns.insert(0, cumulative_reward)
        returns = torch.tensor(returns).to(self.device)

        # Only normalize if we have more than one element to avoid std() warning
        if returns.numel() > 1:
            epsilon = 1e-8  # Small constant to avoid division by zero
            returns_std = returns.std()
            if not torch.isnan(returns_std) and returns_std >= epsilon:
                returns = (returns - returns.mean()) / (returns_std + epsilon)

        #I had conceptual trouble with normalizing the reward by an average, because it seemed to me since we're adding more rewards for earlier timesteps, the cumulative reward for earlier times would be a lot larger. But need to consider dicount facotr.
        # Future rewards contribute significantly to the cumulative return, so earlier timesteps will likely have larger returns.
        #if gamma is close to 0, future rewards have little influence, and the return at each timestep will closely resemble the immediate reward, meaning the pattern might not be as clear.
        return returns

    #The advantage is calculated as the difference between the value predicted by the critic and the expected return from the actions chosen by the actor according to the policy.
    def calculate_advantages(self, returns, values):
        advantages = returns - values

        # Only normalize if we have more than one element to avoid std() warning
        if advantages.numel() > 1:
            epsilon = 1e-8
            advantages_std = advantages.std()
            if not torch.isnan(advantages_std) and advantages_std >= epsilon:
                advantages = (advantages - advantages.mean()) / (advantages_std + epsilon)

        return advantages

    #The standard policy gradient loss is calculated as the product of the policy action probabilities and the advantage function
    #The standard policy gradietn loss cannot make corrections for abrupt policy changes. The surrogate loss modifies the standard loss to restrict the amount the policy can change in each iteration.
    #The surrogate loss is the minimum of (policy ratio X advantage function) and (clipped value of policy ratio X advantage function) where the policy ratio is between the action probabilities according to the old versus new policies and clipping restricts the value to a region near 1.

    def calculate_surrogate_loss(self, actions_log_probability_old, actions_log_probability_new, advantages):
        advantages = advantages.detach()
        # creates a new tensor that shares the same underlying data as the original tensor but breaks the computation graph. This means:
        # The new tensor is treated as a constant with no gradients.
        # Any operations involving this tensor do not affect the gradients of earlier computations in the graph.

        #If the advantages are not detached, the backpropagation of the loss computed using the surrogate_loss would affect both the actor and the critic networks
        # The surrogate loss is meant to update only the policy (actor).
        # Allowing gradients to flow back through the advantages would inadvertently update the critic, potentially disrupting its learning process.

        policy_ratio  = (actions_log_probability_new - actions_log_probability_old).exp()
        surrogate_loss_1 = policy_ratio*advantages
        surrogate_loss_2 = torch.clamp(policy_ratio, min =1.0-self.epsilon, max = 1.0+self.epsilon)*advantages
        surrogate_loss=torch.min(surrogate_loss_1, surrogate_loss_2)
        return surrogate_loss

    #TRAINING THE AGENT
    #Policy loss is the sum of the surrogate loss and the entropy bonus. It is used to update the actor (policy network)
    #Value loss is based on the difference between the value predicted by the critic and the returns (cumulative reward) generated by the policy. This loss is used to update the critic (value network) to make predictions more accurate.

    def calculate_losses(self, surrogate_loss, entropy, returns, value_pred):
        entropy_bonus = self.entropy_coefficient*entropy
        policy_loss = -(surrogate_loss+entropy_bonus).sum()
        value_loss = torch.nn.functional.smooth_l1_loss(returns, value_pred).sum() #helps to smoothen the loss function and makes it less sensitive to outliers.
        return policy_loss, value_loss

    def init_training(self):
        #create a set of buffers as empty arrays. To be used during training to store information
        states = []
        actions = []
        actions_log_probability = []
        values = []
        rewards = []
        done = False
        episode_reward = 0
        return states, actions, actions_log_probability, values, rewards, done, episode_reward

    def forward_pass(self):#this is just the training function (might just want to rename it)
        # # === DETAILED OBJECT ANALYSIS ===
        # import psutil
        # import gc

        # if not hasattr(self, '_episode_counter'):
        #     self._episode_counter = 0
        # self._episode_counter += 1

        # mem_mb = psutil.Process().memory_info().rss / 1024 / 1024

        # # Get ALL objects with "Network" in their type name
        # network_objects = [obj for obj in gc.get_objects() if 'network' in str(type(obj)).lower()]

        # print(f"\n=== EPISODE {self._episode_counter} OBJECT ANALYSIS ===")
        # print(f"Memory: {mem_mb:.1f}MB")
        # print(f"Total objects with 'network' in type: {len(network_objects)}")

        # # Count by exact type
        # type_counts = {}
        # for obj in network_objects:
        #     obj_type = str(type(obj))
        #     type_counts[obj_type] = type_counts.get(obj_type, 0) + 1

        # # Print breakdown
        # for obj_type, count in type_counts.items():
        #     print(f"  {obj_type}: {count}")

        # # Show actual PyPSA Network objects specifically
        # actual_networks = [obj for obj in gc.get_objects() if type(obj).__name__ == 'Network' and 'pypsa' in str(type(obj))]
        # print(f"Actual PyPSA Network objects: {len(actual_networks)}")

        # if len(actual_networks) <= 5:  # Only print if reasonable number
        #     for i, net in enumerate(actual_networks):
        #         print(f"  Network {i+1}: {id(net)} - {type(net)}")

        # network_id = id(self.env.network) if hasattr(self.env, 'network') else None
        # print(f"Current env.network ID: {network_id}")
        # print("=" * 50)
        # # === END ANALYSIS ===

        # Reset environment with seed
        if self.seed is not None:
            state, _ = self.env.reset(seed=self.seed)
        else:
            state, _ = self.env.reset()

        states, actions, actions_log_probability, values, rewards, done, episode_reward = self.init_training()

        # Add this line to track violations
        total_violations = 0

        # # Create fresh network for each episode to avoid memory corruption
        # fresh_network = create_pypsa_network()
        # self.env.network = fresh_network

        state, _ = self.env.reset()  # Gymnasium format returns (obs, info)

        self.model.train() # Set model to training mode

        while True:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            states.append(state_tensor)

            # Get action predictions and values
            action_mean, value_pred = self.model(state_tensor)



            # Split actor output into alpha and beta parameters
            action_dim = self.env.action_space.shape[0]
            alpha_raw, beta_raw = torch.split(action_mean, action_dim, dim=-1)

            # Ensure alpha, beta > 1 for well-behaved Beta distribution
            alpha = torch.nn.functional.softplus(alpha_raw) + 1.0
            beta = torch.nn.functional.softplus(beta_raw) + 1.0

            # Create Beta distribution for continuous actions in [0,1]
            dist = torch.distributions.Beta(alpha, beta)
            action = dist.sample()

            # No clamping needed - Beta distribution naturally outputs [0,1]
            action_clamped = action

            log_prob_action = dist.log_prob(action).sum(dim=-1)  # Sum over action dimensions

            # Step environment with numpy action
            action_np = action_clamped.detach().cpu().numpy().flatten()
            state, reward, terminated, truncated, info = self.env.step(action_np)
            done = terminated or truncated

            #accumulate violations for the epsiode
            total_violations += sum(info['constraint_violations'].values())

            actions.append(action_clamped)
            actions_log_probability.append(log_prob_action)
            values.append(value_pred)
            rewards.append(reward)
            episode_reward += reward

            if done:
                break

        states=torch.cat(states).to(self.device)#converts the list of individual states into a sinlem tensor that is necessary for later processing
        #Creates a single tensor with dimensions like (N, state_dim), where: N is the number of states collected in the episode; state_dim is the dimensionality of each state.
        #torch.cat() expects a sequence (e.g. list or tuple) of PyTorch tensors as input.
        actions=torch.cat(actions).to(self.device)
        #Note that, in the loop, both state and action are PyTorch tensors so that states and actions are both lists of PyTorch tensors
        actions_log_probability=torch.cat(actions_log_probability).to(self.device)
        values=torch.cat(values).squeeze(-1).to(self.device)# .squeeze removes a dimension of size 1 only from tensor at the specified position, in this case, -1, the last dimesion in the tensor. Note that .squeeze() does not do anything if the size of the dimension at the specified potision is not 1.
        # print(f"rewards NaNs: {torch.isnan(torch.tensor(rewards, dtype=torch.float32)).any()}")
        # print(f"values NaNs: {torch.isnan(torch.tensor(values, dtype=torch.float32)).any()}")
        returns = self.calculate_returns(rewards)
        advantages = self.calculate_advantages(returns, values)

        # print(f"Returns NaNs: {torch.isnan(returns).any()}")
        # print(f"advantages NaNs (after calculation): {torch.isnan(advantages).any()}")

        return episode_reward, states, actions, actions_log_probability, advantages, returns, total_violations


    def update_policy(self,
            states,
            actions,
            actions_log_probability_old,
            advantages,
            returns):
        #print(f"Returns NaNs: {torch.isnan(returns).any()}")
        total_policy_loss = 0
        total_value_loss = 0
        actions_log_probability_old = actions_log_probability_old.detach()
        actions=actions.detach()

        # print(f"Returns NaNs: {torch.isnan(returns).any()}")
        # print(f"advantages NaNs (after calculation): {torch.isnan(advantages).any()}")


        #detach() is used to remove the tensor from the computation graph, meaning no gradients will be calculated for that tensor when performing backpropagation.
        #In this context, it's used to ensure that the old actions and log probabilities do not participate in the gradient computation during the optimization of the policy, as we want to update the model based on the current policy rather than the old one.
        #print(type(states), type(actions),type(actions_log_probability_old), type(advantages), type(returns))
        training_results_dataset= TensorDataset(
                states,
                actions,
                actions_log_probability_old,
                advantages,
                returns) #TensorDataset class expects all the arguments passed to it to be tensors (or other compatible types like NumPy arrays, which will be automatically converted to tensor
        batch_dataset = DataLoader(
                training_results_dataset,
                batch_size=self.batch_size,
                shuffle=False)
        #creates a DataLoader instance in PyTorch, which is used to load the training_results_dataset in batches during training.
        #batch_size defines how many samples will be included in each batch. The dataset will be divided into batches of size BATCH_SIZE. The model will then process one batch at a time, rather than all of the data at once,
        #shuffle argument controls whether or not the data will be shuffled before being split into batches.
        #Because shuffle is false, dataloader will provide the batches in the order the data appears in training_results_dataset. In this case, the batches will be formed from consecutive entries in the dataset, and the observations will appear in the same sequence as they are stored in the dataset.
        for _ in range(self.PPO_steps):
            for batch_idx, (states,actions,actions_log_probability_old, advantages, returns) in enumerate(batch_dataset):
                #get new log prob of actions for all input states
                action_mean, value_pred = self.model(states)
                value_pred = value_pred.squeeze(-1)

                # For continuous actions with Beta distribution
                action_dim = self.env.action_space.shape[0]
                alpha_raw, beta_raw = torch.split(action_mean, action_dim, dim=-1)

                # Ensure alpha, beta > 1 for well-behaved Beta distribution
                alpha = torch.nn.functional.softplus(alpha_raw) + 1.0
                beta = torch.nn.functional.softplus(beta_raw) + 1.0

                probability_distribution_new = torch.distributions.Beta(alpha, beta)
                entropy = probability_distribution_new.entropy().sum(dim=-1)

                #estimate new log probabilities using old actions
                actions_log_probability_new = probability_distribution_new.log_prob(actions).sum(dim=-1)
                # # Check for NaN or Inf in log probabilities
                # if torch.isnan(actions_log_probability_old).any() or torch.isinf(actions_log_probability_old).any():
                #     print("NaN or Inf detected in actions_log_probability_old!")
                #     return  # You can return or handle this case as needed

                # if torch.isnan(actions_log_probability_new).any() or torch.isinf(actions_log_probability_new).any():
                #     print("NaN or Inf detected in actions_log_probability_new!")
                #     return  # You can return or handle this case as needed

                # print(f"actions_log_probability_old NaNs: {torch.isnan(actions_log_probability_old).any()}")
                # print(f"actions_log_probability_new NaNs: {torch.isnan(actions_log_probability_new).any()}")
                # print(f"advantages NaNs: {torch.isnan(advantages).any()}")

                surrogate_loss = self.calculate_surrogate_loss(
                    actions_log_probability_old,
                    actions_log_probability_new,
                    advantages
                )

                # print(f"Surrogate Loss NaNs: {torch.isnan(surrogate_loss).any()}")
                # print(f"Entropy NaNs: {torch.isnan(entropy).any()}")
                # print(f"Returns NaNs: {torch.isnan(returns).any()}")
                # print(f"Value Predictions NaNs: {torch.isnan(value_pred).any()}")

                policy_loss, value_loss = self.calculate_losses(
                    surrogate_loss,
                    entropy,
                    returns,
                    value_pred
                )
                self.optimizer.zero_grad() #clear existing gradietns in the optimizer (so that these don't propagate accross multiple .backward(). Ensures each optimization step uses only the gradients computed during the current batch.

                # Skip backward pass if loss is NaN
                if torch.isnan(policy_loss).any():
                    print("NaN detected in policy_loss - skipping backward pass!")
                    continue
                if torch.isnan(value_loss).any():
                    print("NaN detected in value_loss - skipping backward pass!")
                    continue

                policy_loss.backward() #computes gradients for policy_loss with respect to the agent's parameters
                # #Check for NaN gradients after policy_loss backward
                # for param in self.model.parameters():
                #     if param.grad is not None:  # Check if gradients exist for this parameter
                #         if torch.isnan(param.grad).any():
                #             print("NaN gradient detected in policy_loss!")
                # #             return
                value_loss.backward()
                # Check for NaN gradients after value_loss backwardor param in self.model.parameters():
                # for param in self.model.parameters():
                #     if param.grad is not None:  # Check if gradients exist for this parameter
                #         if torch.isnan(param.grad).any():
                #             print("NaN gradient detected in value_loss!")
                #             return

                self.optimizer.step()
                #The update step is based on the learning rate and other hyperparameters of the optimizer
                # The parameters of the agent are adjusted to reduce the policy and value losses.
                total_policy_loss += policy_loss.item() #accumulate the scalar value of the policy loss for logging/ analysis
                #policy_loss.item() extracts the numerical value of the loss tensor (detaching it from the computational graph).
                #This value is added to total_policy_loss to compute the cumulative loss over all batches in the current PPO step.
                #Result: tracks the total policy loss for the current training epoch
                # The loss over the whole dataset is the sum of the losses over all batches.
                #The training dataset is split into batches during the training process. Each batch represents a subset of the collected training data from one episode.
                # Loss calculation is performed for each batch (policy loss and value loss)
                # for each batch, gradients are calculated with respect to the total loss for that batch and the optimizer then updates the network parameters using these gradients.
                # this is because the surrogate loss is only calculated over a single batch of data
                #look at the formula for surrogate loss.
                # It is written in terms of an expectation ˆ Et[. . .] that indicates the empirical average over a finite batch of samples.
                # This means you have collected a set of data (time steps) from the environment, and you're averaging over these data points. The hat symbol implies you're approximating the true expectation with a finite sample of data from the environment. This empirical average can be computed as the mean of values from the sampled transitions
                # the expectation is taken over all the data you've collected
                #If you're training with multiple batches (i.e., collecting data in chunks), then you can think of the expectation as being computed over each batch.
                #The overall expectation can indeed be seen as the sum of expectations computed for each batch, but The expectation of the sum is generally not exactly equal to the sum of the expectations unless the samples are independent, but in practical reinforcement learning algorithms, it's typically a good enough approximation
                #For samples to be independent, the outcome of one sample must not provide any information about the outcome of another. Specifically, in the context of reinforcement learning, this means that the states, actions, rewards, and subsequent states observed in different time steps or different episodes should be independent of each other.
                total_value_loss += value_loss.item()
                #Notice that we are calculating an empirical average, which is already an approximation on the true value (the true expectation would be the average over an infinite amount of data, and the empirical average is the average over the finite amount of data that we have collected).
                #But furthermore, we are approximating even the empirical average istelf. The empirical average is the average over all our collected datal, but here we actually batch our data, calculate average over each batch and then sum these averages, which is not exaclty equal to the average of the sums (but is a decent approximation).
        return total_policy_loss / self.PPO_steps, total_value_loss / self.PPO_steps

    def train(self):
        train_rewards = []
        # test_rewards = []
        # policy_losses = []
        # value_losses = []
        #lens = []

        for episode in range(1, self.max_episodes + 1):
            # Perform a forward pass and collect experience
            train_reward, states, actions, actions_log_probability, advantages, returns, violations = self.forward_pass()

            # Update the policy using the experience collected
            policy_loss, value_loss = self.update_policy(
                states,
                actions,
                actions_log_probability,
                advantages,
                returns)
            # test_reward = self.evaluate()

            # # Visualize the environment if it supports rendering (currently this is done once each episode - might want to change to once every multiple of episodes)
            # if hasattr(self.env, "render") and callable(getattr(self.env, "render", None)):
            #   self.env.render()

            # Log the results
            # policy_losses.append(policy_loss)
            # value_losses.append(value_loss)
            train_rewards.append(train_reward)
            # # run these when back online
            # self.run["policy_loss"].log(policy_loss)
            # self.run["value_loss"].log(value_loss)
            self.run["train_reward"].log(train_reward)
            self.run["total_violation"].log(violations)

            # Calculate the mean of recent rewards and losses for display
            mean_train_rewards = np.mean(train_rewards[-self.n_trials:])
            #mean_test_rewards = np.mean(test_rewards[-self.n_trials:])
            # mean_abs_policy_loss = np.mean(np.abs(policy_losses[-self.n_trials:]))
            # mean_abs_value_loss = np.mean(np.abs(value_losses[-self.n_trials:]))

            # Print results at specified intervals
            if episode % self.print_interval == 0:
                print(f'Episode: {episode:3} | \
                    Train Rewards: {train_reward:3.1f} \
                    Violations: {violations}\
                    Mean Train Rewards: {mean_train_rewards:3.1f}' )
                    # \
                    # | Mean Abs Policy Loss: {mean_abs_policy_loss:2.2f} \
                    # | Mean Abs Value Loss: {mean_abs_value_loss:2.2f} ')



                                    # | Mean Test Rewards: {mean_test_rewards:3.1f} \
                                    #| "Episode Len: {np.mean(lens[-self.n_trials:])}



            # # Check if reward threshold is reached
            # if mean_test_rewards >= self.reward_threshold:
            #     print(f'Reached reward threshold in {episode} episodes')
            #     break
        # Check if the environment has a close method before calling it
        # if hasattr(self.env, "close") and callable(getattr(self.env, "close", None)):
        #   self.env.close() #Close environment visualisation after training is done.
        return train_rewards

def plot_train_rewards(train_rewards, reward_threshold):
    plt.figure(figsize=(12, 8))
    plt.plot(train_rewards, label='Training Reward')
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Training Reward', fontsize=20)
    plt.hlines(reward_threshold, 0, len(train_rewards), color='y')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

def plot_test_rewards(test_rewards, reward_threshold):
    plt.figure(figsize=(12, 8))
    plt.plot(test_rewards, label='Testing Reward')
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Testing Reward', fontsize=20)
    plt.hlines(reward_threshold, 0, len(test_rewards), color='y')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

def plot_losses(policy_losses, value_losses):
    plt.figure(figsize=(12, 8))
    plt.plot(value_losses, label='Value Losses')
    plt.plot(policy_losses, label='Policy Losses')
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Loss', fontsize=20)
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

In [11]:
import ipdb
class Env2Gen1LoadConstrReplacement(Env2Gen1LoadConstr):
    """
    Environment using the Replacement reward method instead of Summation.

    Inherits from Env2Gen1LoadConstr but modifies the reward calculation
    to implement the replacement method from the RL-OPF paper.
    """

    def __init__(self,network_file, episode_length_factor=None, constraint_penalty_factor=100, offset_k=2500, input_dir="var_constraint_map"):
        """
        Initialize the replacement reward environment.

        Parameters:
        -----------
        network_file : str
            Path to the PyPSA network file
        episode_length : int, optional
            Length of episodes (defaults to total snapshots)
        constraint_penalty_factor : float
            Penalty factor for constraint violations
        offset_k : float
            Offset value for replacement reward method
        input_dir : str
            Directory containing constraint mappings
        """
        # Call parent constructor - this will initialize all base attributes
        super().__init__(
            network_file=network_file,
            episode_length_factor=episode_length_factor,
            constraint_penalty_factor=constraint_penalty_factor,
            input_dir=input_dir
        )

        # Add replacement-specific attributes
        self.offset_k = offset_k
        self.reward_method = "replacement"

        # Store initialization parameters for offset calculation
        self.k_method = "mean"  # Default method for k calculation
        self.k_samples = 1000  # Default number of samples for k calculation

    def calculate_constrained_reward(self):
        """
        Calculate reward using replacement method with dynamic constraint checking.

        Replacement method:
        - If all constraints satisfied: return -J(s) + k
        - If constraints violated: return -P(s)
        """
        try:
            # Get base reward from objective function (negative for minimization)
            base_reward = self._calculate_reward()
            # Get constraint violations using the same logic as the parent class
            # But we need to implement it here directly instead of calling super()
            current_snapshot = self.network.snapshots[self.snapshot_idx]

            # Initialize constraint tracking
            constraint_results = {
                'all_satisfied': True,
                'violations': {},
                'total_violation': 0.0,
                'violations_by_group': {}
            }

            # 1. Check slack generator constraints
            slack_generators = self.network.generators[self.network.generators.control == "Slack"].index
            if not slack_generators.empty:
                for gen_name in slack_generators:
                    # Get actual power output after power flow
                    p_actual = self.network.generators_t.p.loc[current_snapshot, gen_name]

                    # Get limits
                    p_min = self.network.generators.loc[gen_name, 'p_min_pu'] * self.network.generators.loc[gen_name, 'p_nom']
                    p_max = self.network.generators.loc[gen_name, 'p_max_pu'] * self.network.generators.loc[gen_name, 'p_nom']

                    # Check lower bound
                    if p_actual < p_min:
                        violation = float(p_min - p_actual)
                        constraint_name = f"Generator-slack-p-lower[snapshot={current_snapshot},Generator={gen_name}]"
                        constraint_results['violations'][constraint_name] = violation
                        constraint_results['total_violation'] += violation
                        constraint_results['all_satisfied'] = False

                    # Check upper bound
                    if p_actual > p_max:
                        violation = float(p_actual - p_max)
                        constraint_name = f"Generator-slack-p-upper[snapshot={current_snapshot},Generator={gen_name}]"
                        constraint_results['violations'][constraint_name] = violation
                        constraint_results['total_violation'] += violation
                        constraint_results['all_satisfied'] = False

            # 2. Check line flow constraints (CORRECTED)
            for line_name in self.network.lines.index:
                # Get line parameters
                s_nom = self.network.lines.loc[line_name, 's_nom']
                s_max_pu = 1.0  # Default, or get from lines_t.s_max_pu if it exists

                # Calculate active power limit (this is what PyPSA's linear constraints check)
                s_max = s_max_pu * s_nom

                # Get active power flow from the linear power flow
                # In PyPSA's linear formulation, this is the 's' variable value
                p0 = abs(self.network.lines_t.p0.loc[current_snapshot, line_name])

                # Check if active power flow exceeds limit
                if p0 > s_max:
                    violation = float(p0 - s_max)
                    constraint_name = f"Line-fix-s-upper[snapshot={current_snapshot},Line={line_name}]"
                    constraint_results['violations'][constraint_name] = violation
                    constraint_results['total_violation'] += violation
                    constraint_results['all_satisfied'] = False

            # Apply replacement method
            if constraint_results['all_satisfied']:
                # All constraints satisfied: return optimization reward + offset k
                constrained_reward = base_reward + self.offset_k
            else:
                # Constraints violated: return only penalty (negative)
                total_violation = float(constraint_results['total_violation'])
                constrained_reward = -self.penalty_factor * total_violation

            # Ensure reward is a scalar
            if hasattr(constrained_reward, '__len__'):
                constrained_reward = float(constrained_reward)

            return constrained_reward, constraint_results

        except Exception as e:
            print(f"Error calculating replacement reward: {e}")
            # Fall back to base reward on error
            return self._calculate_reward(), {
                'all_satisfied': True,
                'violations': {},
                'total_violation': 0.0
            }

    def get_reward_method_info(self):
        """
        Get information about the reward method being used.

        Returns:
        --------
        dict: Information about the reward method
        """
        return {
            'method': 'replacement',
            'offset_k': self.offset_k,
            'k_method': self.k_method,
            'k_samples': self.k_samples,
            'penalty_factor': self.penalty_factor
        }

# network_file_path= "/Users/antoniagrindrod/Documents/pypsa-earth_project/pypsa-earth-RL/networks/elec_s_10_ec_lc1.0_1h.nc"
# input_dir="/Users/antoniagrindrod/Documents/pypsa-earth_project/pypsa-earth-RL/RL/var_constraint_map"
# replacement_reward_offset=calculate_offset_k_initialization(network_file=network_file_path, input_dir=input_dir)

In [12]:
# # Add this to your main function or create a new sweep script

# import time
# from datetime import datetime
# import os # Import os
# from google.colab import drive
# drive.mount('/content/drive')

# def run_sweep_replacement():
#     # Base parameters (non-swept parameters)
#     base_params = {
#         "optimizer_name": "Adam",
#         "MAX_EPISODES": 10000,
#         "PRINT_INTERVAL": 100,  # Print less frequently during sweep
#         "N_TRIALS": 8,
#         "DROPOUT": 0,
#         "network_file": "elec_s_10_ec_lc1.0_1h.nc",
#         "input_dir": "snapshot_dicts"
#     }

#     # Parameters to sweep (key parameters that affect learning)
#     sweep_params = {
#         "LEARNING_RATE": [1e-4],
#         "EPSILON": [0.1],
#         "ENTROPY_COEFFICIENT": [0.01],
#         "HIDDEN_DIMENSIONS": [32],
#         "PPO_STEPS": [2],
#         "BATCH_SIZE": [64],
#         "DISCOUNT_FACTOR": [0.99],
#         "constraint_penalty_factor":[0,100],
#         "episode_length_factor": [1],
#         "env_class": ["Env2Gen1LoadConstr", "Env2Gen1LoadConstrReplacement"] #Will need to change this if you change the env class names
#     }

#     # Construct the full path to the network file in Google Drive
#     gdrive_base = '/content/drive/MyDrive/Colab_Notebooks/networks_1_year_connected'
#     network_file_path = os.path.join(gdrive_base, base_params["network_file"]) # Use os.path.join for path construction
#     gdrive_base2='/content/drive/MyDrive/Colab_Notebooks/'
#     input_dir_path=os.path.join(gdrive_base2, base_params["input_dir"])

#     # Check if the network file exists before proceeding
#     if not os.path.exists(network_file_path):
#         print(f"Error: Network file not found at {network_file_path}")
#         return # Exit the function if the file is not found

#     # Seeds to use for each configuration
#     #seeds = [42, 123, 7]  # Using 3 seeds for statistical significance
#     seeds = [42]
#     #seeds= [123, 7]

#     # Generate a subset of combinations to keep the sweep manageable
#     # We'll use a more focused approach rather than a full grid search

#     # Add some random combinations to explore the space more broadly
#     config_seed=42
#     import random
#     random.seed(config_seed)  # For reproducible random configs

#     #after setting the seed, perform sampling to determine k and store the result

#     num_random_configs = 1  # Adjust based on how many total runs you want
#     random_configs = []

#     for _ in range(num_random_configs):
#         config = {param: random.choice(values) for param, values in sweep_params.items()}
#         random_configs.append(config)

#     # Combine priority and random configs
#     all_configs = random_configs

#     # Print sweep summary
#     print(f"Running sweep with {len(all_configs)} configurations and {len(seeds)} seeds")
#     print(f"Total runs: {len(all_configs) * len(seeds)}")

#     # Run all configurations
#     for seed in seeds:
#         # Set all random seeds
#         random.seed(seed)
#         np.random.seed(seed)
#         torch.manual_seed(seed)
#         if torch.cuda.is_available():
#             torch.cuda.manual_seed(seed)
#             torch.cuda.manual_seed_all(seed)

#         #after setting the seed, perform sampling to determine k and store the result
#         replacement_reward_offset=calculate_offset_k_initialization(network_file=network_file_path, input_dir=input_dir_path)
#         for config_idx, config in enumerate(all_configs):
#               # Create a unique run ID
#             run_id = f"sweep_{datetime.now().strftime('%Y%m%d')}_{config_idx}_{seed}"

#             my_api="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZDg3ZjNlYi04MWI3LTQ1ODctOGIxNS1iNTY3ZjgzMGYzMzYifQ=="

#             # Initialize Neptune run
#             run = neptune.init_run(
#                 project='EnergyGridRL/elec-s-10-ec-lc10-1h-Dispatch',
#                 api_token=my_api,
#                 name=f"Sweep-Config{config_idx}-Seed{seed}",
#                 tags=["hyperparameter_sweep"]
#             )

#             # Combine base params with this config
#             params = {**base_params, **config}
#             params["SEED"] = seed

#             # Log all parameters
#             for key, value in params.items():
#                 run[f"parameters/{key}"] = value

#             run["env_class"]=params["env_class"]
#             print(f"\n{'='*50}")
#             print(f"Starting run {config_idx+1}/{len(all_configs)}, seed {seed}")
#             print(f"Parameters: {params}")
#             print(f"{'='*50}\n")

#             run["results/baseline_reward"]= params["episode_length_factor"]*evaluate_baseline_reward(network_file=network_file_path, env=env,input_dir=input_dir_path) # Pass network_file_path

#             try:

#                 # Create environment and agent
#                 if params["env_class"]=="Env2Gen1LoadConstr":
#                     env = Env2Gen1LoadConstr(network_file=network_file_path, episode_length_factor=params["episode_length_factor"], constraint_penalty_factor=params["constraint_penalty_factor"],input_dir=input_dir_path) # Pass the full input_dir_path
#                 elif params["env_class"]=="Env2Gen1LoadConstrReplacement":
#                     env = Env2Gen1LoadConstrReplacement(network_file=network_file_path, episode_length_factor=params["episode_length_factor"], constraint_penalty_factor=params["constraint_penalty_factor"], offset_k=replacement_reward_offset, input_dir=input_dir_path) # Pass the full input_dir_path
#                 env.seed(seed)

#                 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#                 agent = PPO_agent(
#                     env=env,
#                     run=run,
#                     device=device,
#                     hidden_dimensions=params["HIDDEN_DIMENSIONS"],
#                     dropout=params["DROPOUT"],
#                     discount_factor=params["DISCOUNT_FACTOR"],
#                     optimizer_name=params["optimizer_name"],
#                     max_episodes=params["MAX_EPISODES"],
#                     print_interval=params["PRINT_INTERVAL"],
#                     PPO_steps=params["PPO_STEPS"],
#                     n_trials=params["N_TRIALS"],
#                     epsilon=params["EPSILON"],
#                     entropy_coefficient=params["ENTROPY_COEFFICIENT"],
#                     learning_rate=params["LEARNING_RATE"],
#                     batch_size=params["BATCH_SIZE"],
#                     seed=seed
#                 )

#                 run["replacement_reward"].log(replacement_reward_offset)

#                 # Train the agent
#                 train_rewards = agent.train()

#                 # Log final performance metrics
#                 run["results/final_reward"] = train_rewards[-1]
#                 run["results/mean_last_100_reward"] = np.mean(train_rewards[-100:])
#                 run["results/best_reward"] = np.max(train_rewards)

#             except Exception as e:
#                 print(f"Error in run: {e}")
#                 run["results/error"] = str(e)

#             # Close the Neptune run
#             run.stop()

#             # Small delay to avoid API rate limits
#             time.sleep(1)

In [14]:
# # Add this to your main function or create a new sweep script

# import time
# from datetime import datetime

# def run_sweep_replacement():
#     # Base parameters (non-swept parameters)
#     base_params = {
#         "optimizer_name": "Adam",
#         "MAX_EPISODES": 1000,
#         "PRINT_INTERVAL": 10,  # Print less frequently during sweep
#         "N_TRIALS": 100,
#         "DROPOUT": 0,
#         "network_file": "elec_s_5_ec_lcopt_3h.nc",
#     }

#     # Parameters to sweep (key parameters that affect learning)
#     sweep_params = {
#         "LEARNING_RATE": [1e-4, 3e-4, 1e-3, 3e-3],
#         "EPSILON": [0.1, 0.2, 0.3],
#         "ENTROPY_COEFFICIENT": [0.01, 0.05, 0.1],
#         "HIDDEN_DIMENSIONS": [32, 64, 128],
#         "PPO_STEPS": [8, 16],
#         "BATCH_SIZE": [128, 256],
#         "DISCOUNT_FACTOR": [0.95, 0.99],
#         "constraint_penalty_factor":[0,25,50,100],
#         "episode_length": [2,4,6],
#         "env_class": ["Env2Gen1LoadConstr", "Env2Gen1LoadConstrReplacement"] #Will need to change this if you change the env class names
#     }

#     replacement_env=Env2Gen1LoadConstrReplacement() #this is used to calculate the offset k for the replacement reward method

#     # Seeds to use for each configuration
#     #seeds = [42, 123, 7]  # Using 3 seeds for statistical significance
#     #seeds = [42]
#     seeds= [123, 7]

#     # Generate a subset of combinations to keep the sweep manageable
#     # We'll use a more focused approach rather than a full grid search

#     # Priority configurations based on most promising parameter values
#     # Priority configurations based on most promising parameter values
#     priority_configs = [
#         # Config 1: Higher learning rate, exploration focus
#         {"LEARNING_RATE": 1e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":0, "episode_length": 4},

#         # Config 2: Medium learning rate, balanced approach
#         {"LEARNING_RATE": 3e-4, "EPSILON": 0.2, "ENTROPY_COEFFICIENT": 0.05,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":0,"episode_length": 4},

#         # Config 3: Conservative updates, larger network
#         {"LEARNING_RATE": 1e-4, "EPSILON": 0.1, "ENTROPY_COEFFICIENT": 0.01,
#          "HIDDEN_DIMENSIONS": 128, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":0,"episode_length": 4},

#         # Config 4: Aggressive learning, smaller network
#         {"LEARNING_RATE": 3e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 32, "PPO_STEPS": 8, "BATCH_SIZE": 128,
#          "DISCOUNT_FACTOR": 0.95, "constraint_penalty_factor":0,"episode_length": 4},

#         # Config 1: Higher learning rate, exploration focus
#         {"LEARNING_RATE": 1e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":100,"episode_length": 4},

#         # Config 2: Medium learning rate, balanced approach
#         {"LEARNING_RATE": 3e-4, "EPSILON": 0.2, "ENTROPY_COEFFICIENT": 0.05,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":100,"episode_length": 4},

#         # Config 3: Conservative updates, larger network
#         {"LEARNING_RATE": 1e-4, "EPSILON": 0.1, "ENTROPY_COEFFICIENT": 0.01,
#          "HIDDEN_DIMENSIONS": 128, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":100,"episode_length": 4},

#         # Config 4: Aggressive learning, smaller network
#         {"LEARNING_RATE": 3e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 32, "PPO_STEPS": 8, "BATCH_SIZE": 128,
#          "DISCOUNT_FACTOR": 0.95, "constraint_penalty_factor":100,"episode_length": 4},
#     ]

#     # Add some random combinations to explore the space more broadly
#     config_seed=42
#     import random
#     random.seed(config_seed)  # For reproducible random configs

#     #after setting the seed, perform sampling to determine k and store the result

#     num_random_configs = 8  # Adjust based on how many total runs you want
#     random_configs = []

#     for _ in range(num_random_configs):
#         config = {param: random.choice(values) for param, values in sweep_params.items()}
#         random_configs.append(config)

#     # Combine priority and random configs
#     all_configs = priority_configs + random_configs

#     # Print sweep summary
#     print(f"Running sweep with {len(all_configs)} configurations and {len(seeds)} seeds")
#     print(f"Total runs: {len(all_configs) * len(seeds)}")

#     # Run all configurations
#     for seed in seeds:
#         # Set all random seeds
#         random.seed(seed)
#         np.random.seed(seed)
#         torch.manual_seed(seed)
#         if torch.cuda.is_available():
#             torch.cuda.manual_seed(seed)
#             torch.cuda.manual_seed_all(seed)

#         #after setting the seed, perform sampling to determine k and store the result
#         replacement_reward_offset=calculate_offset_k_initialization(env_instance=replacement_env)
#         for config_idx, config in enumerate(all_configs):
#               # Create a unique run ID
#             run_id = f"sweep_{datetime.now().strftime('%Y%m%d')}_{config_idx}_{seed}"

#             my_api="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZDg3ZjNlYi04MWI3LTQ1ODctOGIxNS1iNTY3ZjgzMGYzMzYifQ=="

#             # Initialize Neptune run
#             run = neptune.init_run(
#                 project='EnergyGridRL/PPO-2snapshots-replacement',
#                 api_token=my_api,
#                 name=f"Sweep-Config{config_idx}-Seed{seed}",
#                 tags=["hyperparameter_sweep"]
#             )

#             # Combine base params with this config
#             params = {**base_params, **config}
#             params["SEED"] = seed

#             # Log all parameters
#             for key, value in params.items():
#                 run[f"parameters/{key}"] = value

#             run["env_class"]=params["env_class"]
#             print(f"\n{'='*50}")
#             print(f"Starting run {config_idx+1}/{len(all_configs)}, seed {seed}")
#             print(f"Parameters: {params}")
#             print(f"{'='*50}\n")

#             try:

#                 # Create environment and agent
#                 if params["env_class"]=="Env2Gen1LoadConstr":
#                     env = Env2Gen1LoadConstr(network_file=params["network_file"], episode_length=params["episode_length"], constraint_penalty_factor=params["constraint_penalty_factor"])
#                 elif params["env_class"]=="Env2Gen1LoadConstrReplacement":
#                     env = Env2Gen1LoadConstrReplacement(network_file=params["network_file"], episode_length=params["episode_length"], constraint_penalty_factor=params["constraint_penalty_factor"], offset_k=replacement_reward_offset)
#                 else:
#                     raise ValueError(f"Invalid environment class: {params["env_class"]}")
#                 env.seed(seed)

#                 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#                 agent = PPO_agent(
#                     env=env,
#                     run=run,
#                     device=device,
#                     hidden_dimensions=params["HIDDEN_DIMENSIONS"],
#                     dropout=params["DROPOUT"],
#                     discount_factor=params["DISCOUNT_FACTOR"],
#                     optimizer_name=params["optimizer_name"],
#                     max_episodes=params["MAX_EPISODES"],
#                     print_interval=params["PRINT_INTERVAL"],
#                     PPO_steps=params["PPO_STEPS"],
#                     n_trials=params["N_TRIALS"],
#                     epsilon=params["EPSILON"],
#                     entropy_coefficient=params["ENTROPY_COEFFICIENT"],
#                     learning_rate=params["LEARNING_RATE"],
#                     batch_size=params["BATCH_SIZE"],
#                     seed=seed
#                 )

#                 run["replacement_reward"].log(replacement_reward_offset)

#                 # Train the agent
#                 train_rewards = agent.train()

#                 # Log final performance metrics
#                 run["results/final_reward"] = train_rewards[-1]
#                 run["results/mean_last_100_reward"] = np.mean(train_rewards[-100:])
#                 run["results/best_reward"] = np.max(train_rewards)
#                 run["results/baseline_reward"]= evaluate_baseline_reward(network_file=params["network_file"], env=env, agent=agent)

#             except Exception as e:
#                 print(f"Error in run: {e}")
#                 run["results/error"] = str(e)

#             # Close the Neptune run
#             run.stop()

#             # Small delay to avoid API rate limits
#             time.sleep(1)

In [15]:
# run_sweep_replacement()

In [16]:
# # Performance comparison test with multiple runs for statistical significance
# import time
# import numpy as np

# # Mock network and get_variable_value function for testing
# class MockNetwork:
#     def __init__(self, variable_values):
#         self.variable_values = variable_values
#         self.access_count = 0

# def get_variable_value(network, var_name):
#     """Simplified mock of the get_variable_value function"""
#     network.access_count += 1  # Count how many times this is called
#     # Simulate the overhead of parsing variable names and accessing network data
#     time.sleep(0.001)  # Small delay to simulate parsing overhead
#     return network.variable_values.get(var_name, 0)

# # Create a test function for the current vectorized approach
# def vectorized_approach(network, coeffs_flat, var_indices, variable_names, objective_const):
#     start_time = time.time()

#     # First create the variable values mapping (as in the original code)
#     var_values = {}
#     for var_name in variable_names:
#         value = get_variable_value(network, var_name)
#         var_values[var_name] = value

#     # Current approach using numpy vectorization
#     result = np.sum(coeffs_flat[var_indices] *
#                     [var_values.get(name, 0) for name in variable_names]) + \
#             objective_const

#     end_time = time.time()
#     return result, end_time - start_time, network.access_count

# # Create a test function for the loop-based approach you suggested
# def loop_approach(network, coeffs_flat, var_indices, variable_names, objective_const):
#     start_time = time.time()

#     # Current approach using numpy vectorization
#     result = np.sum(coeffs_flat[var_indices] *
#                     [get_variable_value(network, var_name) for var_name in variable_names]) + \
#             objective_const

#     end_time = time.time()
#     return result, end_time - start_time, network.access_count

# # Run a benchmark with different sizes and multiple trials
# def run_benchmark(sizes=[10,100,1000], num_trials=50):
#     print(f"Performance comparison: Vectorized vs Loop approach (Average of {num_trials} runs)")
#     print("=" * 80)
#     print(f"{'Size':>10} | {'Vectorized (s)':>15} | {'Loop (s)':>15} | {'Speedup':>10} | {'Results Match':>15} | {'API Calls':>10}")
#     print("-" * 80)

#     for size in sizes:
#         # Arrays to store results from multiple trials
#         vec_times = []
#         loop_times = []
#         results_match_count = 0

#         for trial in range(num_trials):
#             # Generate test data
#             coeffs_flat = np.random.random(size * 2)  # Make array larger than needed
#             var_indices = list(range(size))
#             variable_names = [f"var_{i}" for i in var_indices]
#             variable_values = {name: np.random.random() for name in variable_names}
#             objective_const = np.random.random()

#             # Create fresh mock networks for each test to reset counters
#             network1 = MockNetwork(variable_values)
#             network2 = MockNetwork(variable_values)

#             # Run both approaches
#             vec_result, vec_time, vec_calls = vectorized_approach(
#                 network1, coeffs_flat, var_indices, variable_names, objective_const)

#             loop_result, loop_time, loop_calls = loop_approach(
#                 network2, coeffs_flat, var_indices, variable_names, objective_const)

#             # Store results
#             vec_times.append(vec_time)
#             loop_times.append(loop_time)

#             # Check if results match
#             if np.isclose(vec_result, loop_result):
#                 results_match_count += 1

#         # Calculate averages
#         avg_vec_time = np.mean(vec_times)
#         avg_loop_time = np.mean(loop_times)
#         avg_speedup = avg_loop_time / avg_vec_time if avg_vec_time > 0 else float('inf')

#         # Calculate standard deviations for error analysis
#         std_vec_time = np.std(vec_times)
#         std_loop_time = np.std(loop_times)

#         # Results match percentage
#         match_percentage = (results_match_count / num_trials) * 100

#         # Print results
#         print(f"{size:>10} | {avg_vec_time:>12.6f} ±{std_vec_time:.6f} | {avg_loop_time:>12.6f} ±{std_loop_time:.6f} | {avg_speedup:>10.2f}x | {match_percentage:>13.1f}% | {size}/{size}")

# # Run the benchmark with 50 trials
# run_benchmark(num_trials=10)


Performance comparison: Vectorized vs Loop approach (Average of 50 runs)
================================================================================
      Size |  Vectorized (s) |        Loop (s) |    Speedup |   Results Match |  API Calls
--------------------------------------------------------------------------------
        10 |     0.012438 ±0.000475 |     0.012305 ±0.000456 |       0.99x |         100.0% | 10/10
       100 |     0.125578 ±0.010998 |     0.129464 ±0.035359 |       1.03x |         100.0% | 100/100
      1000 |     1.267099 ±0.037849 |     1.271847 ±0.030882 |       1.00x |         100.0% | 1000/1000